In [1]:
import os
gpu_device = 0    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7f1b1cd29120; to 'numba.cuda.cudadrv.driver.Device' at 0x7f1b217ddd30>

In [2]:
import pandas as pd
import re
import random
import numpy as np
import math
import pickle
from sklearn.metrics import classification_report, accuracy_score
from datasets import Dataset,concatenate_datasets,load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%run ../utils/data.ipynb
%run ../utils/Generatives-small_data.ipynb

In [4]:

# Cargar modelo y tokenizador
model_name = "microsoft/biogpt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs,targets,val_data,examples_raw = flatten_and_filter_dataset(ground_truth,report_data)  
inputs2,targets2,val_data2,examples_raw2= flatten_and_filter_dataset_no_prev(ground_truth,report_data) 

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
)

training_args_all = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    # evaluation_strategy="epoch",
    learning_rate=1e-5,  # Increased for better adaptation
    per_device_train_batch_size=8,
    fp16=True,
    gradient_accumulation_steps=2,
    num_train_epochs=2,  # Increased epochs to account for repeated reports
    weight_decay=0.01,  # Prevents overfitting
    save_total_limit=2,
    eval_steps=40,
)
training_args_answer = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    # evaluation_strategy="epoch",
    learning_rate=1e-5,  # Lower for fine-tuning without losing generalization
    fp16=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    
    num_train_epochs=5,  # Shorter fine-tuning stage
    weight_decay=0.01,  # Lower weight decay to preserve learned features
    save_total_limit=2,
)

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
dataset_final=pd.DataFrame.from_dict(inputs,orient='index')
targets=pd.DataFrame.from_dict(targets,orient='index')
val_data=pd.DataFrame.from_dict(val_data,orient='index')
dataset_final.columns=["text"]
targets.columns=["label"]
val_data.columns=["val_data"]
dataset_final=pd.concat([dataset_final, val_data],axis=1)

dataset_final2=pd.DataFrame.from_dict(inputs2,orient='index')
targets2=pd.DataFrame.from_dict(targets2,orient='index')
val_data2=pd.DataFrame.from_dict(val_data2,orient='index')
dataset_final2.columns=["text"]
targets2.columns=["label"]
val_data2.columns=["val_data"]
dataset_final2=pd.concat([dataset_final2, val_data2],axis=1)


In [6]:
accuracies,output_dic=cross_validation_all(dataset_final,targets,training=True,low_beams=False,testing=False)

fold 0
ya existe
                                                                                             val_data  \
001-323-702-20221219-120748.390_age                 Question: does the patient's age appear in the...   
001-323-702-20221219-120748.390_birads              Question: what is the final BI-RADS classifica...   
001-323-702-20221219-120748.390_calcifications_...  Question: does the following breast medical re...   
001-323-702-20221219-120748.390_density_echo        Question: what is the breast density found in ...   
001-323-702-20221219-120748.390_density_mammo       Question: what is the breast density found in ...   
...                                                                                               ...   
046-630-291-20230619-112356_age                     Question: does the patient's age appear in the...   
046-630-291-20230619-112356_family                  Question: does the patient have any family his...   
046-630-291-20230619-112356_history   

Map: 100%|██████████| 405/405 [00:02<00:00, 159.09 examples/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 31.12 MiB is free. Process 796300 has 306.00 MiB memory in use. Including non-PyTorch memory, this process has 15.44 GiB memory in use. Of the allocated memory 15.02 GiB is allocated by PyTorch, and 50.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
accuracies

[0.9952830188679245,
 0.8773584905660378,
 0.9811320754716981,
 0.8548387096774194,
 0.010752688172043012,
 0.010752688172043012,
 0.9946236559139785,
 0.9838709677419355,
 0.978494623655914,
 0.9946236559139785,
 0.19021739130434784,
 0.967741935483871,
 0.9408602150537635,
 0.9623655913978495,
 0.016129032258064516,
 0.989247311827957,
 0.989247311827957,
 0.9354838709677419,
 0.012195121951219513,
 0.2804878048780488,
 0.06097560975609756,
 0.08536585365853659,
 0.024390243902439025,
 0.2804878048780488,
 0.7926829268292683,
 0.9069767441860465]

In [ ]:
first fold
4 epoch 1e-5
Accuracy for age: 1.0000
Accuracy for tipo: 1.0000
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 0.9500
Accuracy for symtomatic: 0.9500
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 0.9474
Accuracy for density_echo: 0.8947
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.9474
Accuracy for nodules_echo_margin: 0.8750
Accuracy for nodules_echo_echogenicity: 1.0000
Accuracy for nodules_echo_location: 0.8750
Accuracy for nodules_echo_size: 0.8750
Accuracy for nodules_echo_known: 0.8750
Accuracy for nodules_echo_stable: 1.0000


2 epoch 5e-6
Accuracy for age: 1.0000
Accuracy for tipo: 1.0000
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.9500
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 0.9474
Accuracy for density_echo: 0.8947
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.9474
Accuracy for nodules_echo_shape: 0.8750
Accuracy for nodules_echo_margin: 0.7500
Accuracy for nodules_echo_echogenicity: 1.0000
Accuracy for nodules_echo_location: 0.5000
Accuracy for nodules_echo_size: 0.8750
Accuracy for nodules_echo_known: 1.0000
Accuracy for nodules_echo_stable: 1.0000

In [ ]:
second fold
Accuracy for age: 1.0000
Accuracy for tipo: 0.9048
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.8824
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.8889
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 1.0000
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.7778
Accuracy for nodules_echo_margin: 0.8000
Accuracy for nodules_echo_echogenicity: 0.8000
Accuracy for nodules_echo_location: 0.7000
Accuracy for nodules_echo_size: 0.8889
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.6000

Accuracy for age: 1.0000
Accuracy for tipo: 0.9048
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 1.0000
Accuracy for symtomatic: 0.9412
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.9444
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 1.0000
Accuracy for lymph_suspicious: 0.9444
Accuracy for nodules_echo_num: 0.6111
Accuracy for nodules_echo_shape: 0.8000
Accuracy for nodules_echo_margin: 0.6000
Accuracy for nodules_echo_echogenicity: 0.8000
Accuracy for nodules_echo_location: 0.5000
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.8000

Accuracy for age: 1.0000
Accuracy for tipo: 0.9524
Accuracy for tecnica: 1.0000
Accuracy for family: 1.0000
Accuracy for history: 0.5882
Accuracy for symtomatic: 0.8235
Accuracy for prosthesis: 1.0000
Accuracy for birads: 0.9444
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 0.8889
Accuracy for lymph_suspicious: 1.0000
Accuracy for nodules_echo_num: 0.7222
Accuracy for nodules_echo_shape: 0.9000
Accuracy for nodules_echo_margin: 0.7000
Accuracy for nodules_echo_echogenicity: 0.6000
Accuracy for nodules_echo_location: 0.6000
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.7778
Accuracy for nodules_echo_stable: 0.6000

In [ ]:
third fold
Accuracy for age: 0.9500
Accuracy for tipo: 0.9500
Accuracy for tecnica: 0.9000
Accuracy for family: 1.0000
Accuracy for history: 0.7222
Accuracy for symtomatic: 0.9444
Accuracy for prosthesis: 1.0000
Accuracy for birads: 1.0000
Accuracy for density_mammo: 1.0000
Accuracy for density_echo: 0.8333
Accuracy for lymph_suspicious: 0.9444
Accuracy for nodules_echo_num: 0.8333
Accuracy for nodules_echo_margin: 0.8889
Accuracy for nodules_echo_echogenicity: 0.8889
Accuracy for nodules_echo_location: 0.7778
Accuracy for nodules_echo_size: 1.0000
Accuracy for nodules_echo_known: 0.9000
Accuracy for nodules_echo_stable: 0.8571

In [8]:
for i,tipo in enumerate(questions):
    print(tipo, accuracies[i])

age 0.9952830188679245
tipo 0.8773584905660378
tecnica 0.9811320754716981
family 0.8548387096774194
history 0.010752688172043012
symtomatic 0.010752688172043012
prosthesis 0.9946236559139785
birads 0.9838709677419355
density_mammo 0.978494623655914
calcifications_benign 0.9946236559139785
density_echo 0.19021739130434784
ganglio_mamo 0.967741935483871
lymph_benign 0.9408602150537635
lymph_suspicious 0.9623655913978495
parenchymal_distortion 0.016129032258064516
simple_cyst 0.989247311827957
ductal_ectasia 0.989247311827957
nodules_echo_num 0.9354838709677419
nodules_echo_description 0.012195121951219513
nodules_echo_shape 0.2804878048780488
nodules_echo_margin 0.06097560975609756
nodules_echo_echogenicity 0.08536585365853659
nodules_echo_location 0.024390243902439025
nodules_echo_size 0.2804878048780488
nodules_echo_known 0.7926829268292683
nodules_echo_stable 0.9069767441860465


In [ ]:
age 0.9952830188679245
tipo 0.9716981132075472
tecnica 0.9858490566037735
family 0.989247311827957
history 0.9301075268817204
symtomatic 0.0
prosthesis 0.9946236559139785
birads 0.967741935483871
density_mammo 0.9516129032258065
density_echo 0.6935483870967742
lymph_suspicious 0.9731182795698925
nodules_echo_num 0.7849462365591398
nodules_echo_size 0.7349397590361446
nodules_echo_known 0.7831325301204819
nodules_echo_stable nan

In [13]:
questions

['age',
 'tipo',
 'tecnica',
 'family',
 'history',
 'symtomatic',
 'prosthesis',
 'birads',
 'density_mammo',
 'calcifications_benign',
 'density_echo',
 'ganglio_mamo',
 'lymph_benign',
 'lymph_suspicious',
 'parenchymal_distortion',
 'simple_cyst',
 'ductal_ectasia',
 'nodules_echo_num',
 'nodules_echo_description',
 'nodules_echo_shape',
 'nodules_echo_margin',
 'nodules_echo_echogenicity',
 'nodules_echo_location',
 'nodules_echo_size',
 'nodules_echo_known',
 'nodules_echo_stable']

In [11]:
model_name="biogpt"+"21e51e57"+"no_copy"+"description"+"16batch"+"final"
for tipo in questions:
    print(tipo)
    with open(f"results_dic_{tipo}/{model_name.split("/")[-1]}.pkl", "wb") as file:
        pickle.dump(output_dic[tipo], file)
   

age
tipo
tecnica
family
history
symtomatic
prosthesis
birads
density_mammo
calcifications_benign
density_echo
ganglio_mamo
lymph_benign
lymph_suspicious
parenchymal_distortion
simple_cyst
ductal_ectasia
nodules_echo_num
nodules_echo_description
nodules_echo_shape
nodules_echo_margin
nodules_echo_echogenicity
nodules_echo_location
nodules_echo_size
nodules_echo_known
nodules_echo_stable


In [9]:
devices = cuda.gpus
dev=devices.current
cuda.select_device(dev.id)
cuda.close()

In [8]:
accuracies 4 epochs 1 e-5
[0.9952830188679245, 0.9764150943396226, 0.9764150943396226]

[0.9952830188679245, 0.9764150943396226, 0.9764150943396226]

In [12]:
acc,acc_std
solo un fallo

(0.9952830188679245, 0.013636363636363624)

In [ ]:
2 epoch eta bikoitza
(0.9811320754716981, 0.026424662271916612)

In [ ]:
con 8 epochs

In [ ]:
con 4 epochs pero añadiendo los datos sin etiqueta.

In [10]:
5 epochs 1e-5
" Important! We do not want biopsy, BAG, post BAG, nodal staging ultrasound or BI-RADS 6 (Cancer patients) reports. Answer 'no' if it is any of these. answer: "
(0.9811320754716981, 0.022268088570756142)

(0.9811320754716981, 0.022268088570756142)